<a href="https://colab.research.google.com/github/jorell1/PyLZJD_Jupyter/blob/master/Implementing_LZJDh%2C_test_on_Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing pyLZJD Clustering Shakespeare

In [3]:
from google.colab import drive
drive.mount("/content/drive")
!pip install git+git://github.com/EdwardRaff/pyLZJD#egg=pyLZJD

Mounted at /content/drive
  Cloning git://github.com/EdwardRaff/pyLZJD to /tmp/pip-install-n24gyf2j/pyLZJD
  Running command git clone -q git://github.com/EdwardRaff/pyLZJD /tmp/pip-install-n24gyf2j/pyLZJD
  Created wheel for pyLZJD: filename=pyLZJD-0.0.2-cp36-none-any.whl size=14687 sha256=2f2b94f14bad5a16e2c9ed34b0fa726703f6de9951a004b636f2c83c11c3d91f
  Stored in directory: /tmp/pip-ephem-wheel-cache-bbs2bwbr/wheels/b1/ab/c2/f1240358b61998fca73ea5f4f2432c8872716c20f767173d9c
Successfully built pyLZJD


In [6]:
!ls drive/My\ Drive/Colab\ Notebooks/WS
dataDir = "drive/My Drive/Colab Notebooks/WS"



AllsWell.txt	    Henry8.txt		  Richard2.txt
AnthonyAndCleo.txt  JuliusCaesar.txt	  Richard3.txt
AsYouLikeIt.txt     KingJohn.txt	  RomeoJuliet.txt
ComedyOfErrors.txt  KingLear.txt	  TamingShrew.txt
Coriolanus.txt	    LoveLabors.txt	  Tempest.txt
Cymbeline.txt	    Macbeth.txt		  Timon.txt
Hamlet.txt	    Measure4Measure.txt   TitusAndronicus.txt
Henry4Part1.txt     MerchantOfVenice.txt  TroiluisCressida.txt
Henry4Part2.txt     MerryWives.txt	  TwelfthNight.txt
Henry5.txt	    MidsummerNights.txt   TwoGentlemenVerona.txt
Henry6Part1.txt     MuchAdo.txt		  WinterTale.txt
Henry6Part2.txt     Othello.txt
Henry6Part3.txt     Pericles.txt


In [7]:
import os, glob, time
from os.path import isfile, join
import numpy as np


from pyLZJD import sim, digest

text_files = glob.glob(join(dataDir, "*.txt"))
#text_files = [f for f in os.listdir(dataDir) if f.endswith('.txt')]
#print(text_files)
n = len(text_files)
result = digest(text_files)
print(text_files)


/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /usr/local/lib/python3.6/dist-packages/pyLZJD/lzjd_cython.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


['drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt', 'drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt', 'drive/My Drive/Colab Notebooks/WS/Coriolanus.txt', 'drive/My Drive/Colab Notebooks/WS/Cymbeline.txt', 'drive/My Drive/Colab Notebooks/WS/Henry4Part1.txt', 'drive/My Drive/Colab Notebooks/WS/Henry4Part2.txt', 'drive/My Drive/Colab Notebooks/WS/Henry6Part1.txt', 'drive/My Drive/Colab Notebooks/WS/Henry6Part2.txt', 'drive/My Drive/Colab Notebooks/WS/Henry6Part3.txt', 'drive/My Drive/Colab Notebooks/WS/Henry8.txt', 'drive/My Drive/Colab Notebooks/WS/JuliusCaesar.txt', 'drive/My Drive/Colab Notebooks/WS/KingJohn.txt', 'drive/My Drive/Colab Notebooks/WS/KingLear.txt', 'drive/My Drive/Colab Notebooks/WS/LoveLabors.txt', 'drive/My Drive/Colab Notebooks/WS/Macbeth.txt', 'drive/My Drive/Colab Notebooks/WS/Measure4Measure.txt', 'drive/My Drive/Colab Notebooks/WS/MerchantOfVenice.txt', 'drive/My Drive/Colab Notebooks/WS/MerryWives.txt', 'drive/My Drive/Colab Notebooks/WS/MidsummerNights.t

Now compute the lzsets.  This needs to be done only once.  The lzset can be thought of as a set of strings such that all the strings occur in the lzset can be concatenedated, in some order and with the given amount of repetiton, to recreate the original document.  Also, each string in the lzset does occur in the document at least once, but if the frequency is zero, the string DOES occur at least once.  It was just not needed for the sake of LZ compression.  

In [8]:
#build a similarity matrix for use in hierarchical clustering
# sim2 will be upper triangular only!
nchoose2 = n*(n-1)//2
print("Number of files is %d" % n)
print("Number of pairs (n choose 2) is %d" % nchoose2)

import os, glob
from os.path import isfile, join

from pyLZJD import sim, digest

text_files = glob.glob(join(dataDir, "*.txt"))
#text_files = [f for f in os.listdir(dataDir) if f.endswith('.txt')]
#print(text_files)
n = len(text_files)
result = digest(text_files)

sim2 = np.zeros(nchoose2)
k = 0
shortestD = 100
f1 = f2 = ""
ishort = jshort = -1

for i in range(n-1):
    print("computing similarity matrix row %d/%d (%s)" % (i+1, n, text_files[i]))
    for j in range(i+1,n):
        theDistance = sim(result[i],result[j])
        sim2[k] = theDistance
        if  theDistance < shortestD:
            shortestD = theDistance
            f1 = text_files[i]
            ishort = i
            f2 = text_files[j]
            jshort = j
            print("Closest files so far, with distance of %f (%d, %d): %s and %s" % 
                  (theDistance, ishort, jshort, f1, f2))
        k += 1

Number of files is 37
Number of pairs (n choose 2) is 666
computing similarity matrix row 1/37 (drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt)
Closest files so far, with distance of 0.179724 (0, 1): drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt and drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt
Closest files so far, with distance of 0.178366 (0, 3): drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt and drive/My Drive/Colab Notebooks/WS/Cymbeline.txt
Closest files so far, with distance of 0.175660 (0, 4): drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt and drive/My Drive/Colab Notebooks/WS/Henry4Part1.txt
Closest files so far, with distance of 0.159683 (0, 14): drive/My Drive/Colab Notebooks/WS/AsYouLikeIt.txt and drive/My Drive/Colab Notebooks/WS/Macbeth.txt
computing similarity matrix row 2/37 (drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt)
Closest files so far, with distance of 0.124657 (1, 2): drive/My Drive/Colab Notebooks/WS/ComedyOfErrors.txt and drive/My D

In [9]:
print("According to pyLZJD, the two closest plays are (%d, %d): %s and %s, with a distance of %f" 
      % (ishort, jshort, f1, f2, shortestD))

According to LZJD, the two closest plays are (33, 34): drive/My Drive/Colab Notebooks/WS/Hamlet.txt and drive/My Drive/Colab Notebooks/WS/TwoGentlemenVerona.txt, with a distance of 0.117904


Granted that Henry 6 parts 2 and 3 are the most similar plays.  What strings do that have in common?  

In [30]:
text_files[11]

'drive/My Drive/Colab Notebooks/WS/KingJohn.txt'

In [31]:
text_files[12]

'drive/My Drive/Colab Notebooks/WS/KingLear.txt'

In [10]:
import os
dataDir = "drive/My Drive/Colab Notebooks/WS"
text_files = [f for f in os.listdir(dataDir) if f.endswith('.txt')]
print(text_files)
n = len(text_files)

['AsYouLikeIt.txt', 'ComedyOfErrors.txt', 'Coriolanus.txt', 'Cymbeline.txt', 'Henry4Part1.txt', 'Henry4Part2.txt', 'Henry6Part1.txt', 'Henry6Part2.txt', 'Henry6Part3.txt', 'Henry8.txt', 'JuliusCaesar.txt', 'KingJohn.txt', 'KingLear.txt', 'LoveLabors.txt', 'Macbeth.txt', 'Measure4Measure.txt', 'MerchantOfVenice.txt', 'MerryWives.txt', 'MidsummerNights.txt', 'MuchAdo.txt', 'Othello.txt', 'Pericles.txt', 'Richard2.txt', 'Richard3.txt', 'TamingShrew.txt', 'Tempest.txt', 'Timon.txt', 'TitusAndronicus.txt', 'TwelfthNight.txt', 'WinterTale.txt', 'AllsWell.txt', 'TroiluisCressida.txt', 'AnthonyAndCleo.txt', 'Hamlet.txt', 'TwoGentlemenVerona.txt', 'Henry5.txt', 'RomeoJuliet.txt']


In [12]:
#read in a bunch of files
buff = [open(dataDir+"/"+f, "rb").read() for f in text_files]

#taking up how much space?
buffSize = 0
for i in range(n):
    buffSize += len(buff[i])
print("Raw data takes up how much space? %d" % (buffSize))

Raw data takes up how much space? 5177104


In [16]:
!cp "drive/My Drive/Colab Notebooks/LZJD.py" .


from LZJD import *

start = time.clock()
nullLZset = lzset(b"")
lzSets = [nullLZset for i in range(n)]

start = time.clock()
for i in range(n):
    print("Building lzset for %s" % (text_files[i]))
    lzSets[i] = lzset(buff[i])
print("Execution time for building LZSets: %f" % (time.clock() - start))

Building lzset for AsYouLikeIt.txt
Building lzset for ComedyOfErrors.txt
Building lzset for Coriolanus.txt
Building lzset for Cymbeline.txt
Building lzset for Henry4Part1.txt
Building lzset for Henry4Part2.txt
Building lzset for Henry6Part1.txt
Building lzset for Henry6Part2.txt
Building lzset for Henry6Part3.txt
Building lzset for Henry8.txt
Building lzset for JuliusCaesar.txt
Building lzset for KingJohn.txt
Building lzset for KingLear.txt
Building lzset for LoveLabors.txt
Building lzset for Macbeth.txt
Building lzset for Measure4Measure.txt
Building lzset for MerchantOfVenice.txt
Building lzset for MerryWives.txt
Building lzset for MidsummerNights.txt
Building lzset for MuchAdo.txt
Building lzset for Othello.txt
Building lzset for Pericles.txt
Building lzset for Richard2.txt
Building lzset for Richard3.txt
Building lzset for TamingShrew.txt
Building lzset for Tempest.txt
Building lzset for Timon.txt
Building lzset for TitusAndronicus.txt
Building lzset for TwelfthNight.txt
Building l

In [17]:
#build a similarity matrix for use in hierarchical clustering
# sim2 will be upper triangular only!
nchoose2 = n*(n-1)//2
print("Number of files is %d" % n)
print("Number of pairs (n choose 2) is %d" % nchoose2)

sim2 = np.zeros(nchoose2)
k = 0
shortestD = 100
f1 = f2 = ""
ishort = jshort = -1

for i in range(n-1):
    print("computing similarity matrix row %d/%d (%s)" % (i+1, n, text_files[i]))
    for j in range(i+1,n):
        theDistance = lzSets[i].lzjd(lzSets[j])
        sim2[k] = theDistance
        if  theDistance < shortestD:
            shortestD = theDistance
            f1 = text_files[i]
            ishort = i
            f2 = text_files[j]
            jshort = j
            print("Closest files so far, with distance of %f (%d, %d): %s and %s" % 
                  (theDistance, ishort, jshort, f1, f2))
        k += 1

Number of files is 37
Number of pairs (n choose 2) is 666
computing similarity matrix row 1/37 (AsYouLikeIt.txt)
Closest files so far, with distance of 0.758639 (0, 1): AsYouLikeIt.txt and ComedyOfErrors.txt
Closest files so far, with distance of 0.753558 (0, 3): AsYouLikeIt.txt and Cymbeline.txt
Closest files so far, with distance of 0.721734 (0, 13): AsYouLikeIt.txt and LoveLabors.txt
Closest files so far, with distance of 0.713828 (0, 19): AsYouLikeIt.txt and MuchAdo.txt
Closest files so far, with distance of 0.710284 (0, 30): AsYouLikeIt.txt and AllsWell.txt
computing similarity matrix row 2/37 (ComedyOfErrors.txt)
computing similarity matrix row 3/37 (Coriolanus.txt)
computing similarity matrix row 4/37 (Cymbeline.txt)
Closest files so far, with distance of 0.709323 (3, 26): Cymbeline.txt and Timon.txt
Closest files so far, with distance of 0.700122 (3, 32): Cymbeline.txt and AnthonyAndCleo.txt
computing similarity matrix row 5/37 (Henry4Part1.txt)
Closest files so far, with dista

In [18]:
print("According to LZJD, the two closest plays are (%d, %d): %s and %s, with a distance of %f" 
      % (ishort, jshort, f1, f2, shortestD))

According to LZJD, the two closest plays are (7, 8): Henry6Part2.txt and Henry6Part3.txt, with a distance of 0.693913
